## Isolated Forest for Detecting Financial Anomalies

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.datasets import make_classification

In [2]:
# Columns to keep
columns = ['amtdb',
 'category',
 'subcategory',
 'fdb_commonname',
 'amtcr',
 'brnid',
 'brnmisc7',
 'altdept',
 'amtbalance',
 'date',
 'department',
 'type',
 'typ_text',]

In [3]:
# Read the file
df = pd.read_excel("~/Desktop/Data/df.xlsx", usecols=columns)
df.head(5)

,brnid,date,type,amtdb,amtcr,amtbalance,department,altdept,typ_text,category,subcategory,fdb_commonname,brnmisc7
0,200,45382,Expense,0.0,10.000000,-10.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC
1,200,45351,Expense,10.0,0.000000,10.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC
2,200,45504,Expense,0.0,155.000000,-155.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC
3,200,45382,Expense,0.0,853.549988,-853.549988,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC
4,200,45504,Expense,0.0,900.000000,-900.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC


In [4]:
# Convert column date into the dates
df['date'] = pd.to_datetime(df['date'], origin='1899-12-30', unit='D')
df.head(5)

,brnid,date,type,amtdb,amtcr,amtbalance,department,altdept,typ_text,category,subcategory,fdb_commonname,brnmisc7
0,200,2024-03-31,Expense,0.0,10.000000,-10.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC
1,200,2024-02-29,Expense,10.0,0.000000,10.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC
2,200,2024-07-31,Expense,0.0,155.000000,-155.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC
3,200,2024-03-31,Expense,0.0,853.549988,-853.549988,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC
4,200,2024-07-31,Expense,0.0,900.000000,-900.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC


In [5]:
# Create columns for year and month from the date column
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [6]:
# drop July 2024 data because it is incomplete
df = df.drop(df[(df['year'] == 2024) & (df['month'] == 7)].index)
df.head(5)

,brnid,date,type,amtdb,amtcr,amtbalance,department,altdept,typ_text,category,subcategory,fdb_commonname,brnmisc7,year,month
0,200,2024-03-31,Expense,0.0,10.000000,-10.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC,2024,3
1,200,2024-02-29,Expense,10.0,0.000000,10.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC,2024,2
3,200,2024-03-31,Expense,0.0,853.549988,-853.549988,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC,2024,3
5,200,2024-03-31,Expense,0.0,55.000000,-55.000000,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC,2024,3
6,200,2024-06-30,Expense,0.0,7138.649902,-7138.649902,Corporate,Used Truck Sales,Expense,Personnel,Employee Benefits,Employee Benefits,Corporate LATC,2024,6


## Anomalies Detections

#### Date Preparation

In [7]:
# Create a copy of the dataframe
df_copy = df.copy()

In [8]:
# Sort the DataFrame by year (descending), brnid, fdb_commonname, and month (ascending)
df_1 = df_copy.sort_values(by=[
    'brnid', 'brnmisc7', 'department', 'altdept', 'typ_text', 'category', 
    'subcategory', 'fdb_commonname', 'year', 'amtbalance', 'month'
])

# Rearrange the columns in the specified order
columns_order = ['date', 'year', 'month', 'brnid', 'brnmisc7', 'department', 'altdept', 'type', 'typ_text', 'category', 'subcategory', 'fdb_commonname', 
                 'amtdb', 'amtcr', 'amtbalance']
df_1 = df_1[columns_order]
df_1 = df_1.reset_index(drop=True)
df_1.head(25)



,date,year,month,brnid,brnmisc7,department,altdept,type,typ_text,category,subcategory,fdb_commonname,amtdb,amtcr,amtbalance
0,2023-07-31,2023,7,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,0.0,6702.0,-6702.0
1,2023-07-31,2023,7,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,0.0,6509.0,-6509.0
2,2023-07-31,2023,7,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,0.0,1798.0,-1798.0
3,2023-07-31,2023,7,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,0.0,1484.0,-1484.0
4,2023-01-31,2023,1,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,245.0,0.0,245.0
5,2023-02-28,2023,2,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,245.0,0.0,245.0
6,2023-04-30,2023,4,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,245.0,0.0,245.0
7,2023-05-31,2023,5,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,245.0,0.0,245.0
8,2023-12-31,2023,12,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,245.0,0.0,245.0
9,2023-06-30,2023,6,200,Corporate LATC,Corporate,Corporate,Expense,Expense,Fixed,Rent / Lease,Rent/Lease,266.0,0.0,266.0


In [9]:
#Group the dataframe with resepct to brnid and fdb_commonname save as df_grouped in data file to do abservation
grouped_df_1 = df_1.groupby(['brnid', 'fdb_commonname']).apply(lambda x: x)
grouped_df_1.head(25)
# df_1.groupby(['brnid', 'fdb_commonname']).apply(lambda x: x).to_excel("~/Desktop/Data/df_grouped.xlsx", index=False) 

/var/folders/ld/1rz8qh_13jz1s4wm8s6pdhgm0000gn/T/ipykernel_79638/223004813.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df_1 = df_1.groupby(['brnid', 'fdb_commonname']).apply(lambda x: x)


date  year  month  brnid  \
brnid fdb_commonname                                                  
200   Building Repair and Maint 5103 2023-02-28  2023      2    200   
                                5104 2023-04-30  2023      4    200   
                                5105 2023-03-31  2023      3    200   
                                5106 2023-10-31  2023     10    200   
                                5107 2023-09-30  2023      9    200   
                                5108 2023-11-30  2023     11    200   
                                5109 2023-03-31  2023      3    200   
                                5110 2023-01-31  2023      1    200   
                                5111 2023-01-31  2023      1    200   
                                5112 2024-04-30  2024      4    200   
                                5113 2024-06-30  2024      6    200   
                                5114 2024-06-30  2024      6    200   
                                5115 2024-06-30  2024      6    200   
                                5116 2024-05-31  2024      5    200   
                                5117 2024-01-31  2024      1    200   
                                5118 2024-04-30  2024      4    200   
                                5266 2024-06-30  2024      6    200   
                                5267 2024-06-30  2024      6    200   
                                5268 2024-06-30  2024      6    200   
      Employee Benefits         394  2023-12-31  2023     12    200   
                                395  2023-12-31  2023     12    200   
                                396  2023-12-31  2023     12    200   
                                397  2023-12-31  2023     12    200   
                                398  2023-12-31  2023     12    200   
                                399  2023-12-31  2023     12    200   

                                            brnmisc7 department    altdept  \
brnid fdb_commonname                                                         
200   Building Repair and Maint 5103  Corporate LATC  Corporate  Corporate   
                                5104  Corporate LATC  Corporate  Corporate   
                                5105  Corporate LATC  Corporate  Corporate   
                                5106  Corporate LATC  Corporate  Corporate   
                                5107  Corporate LATC  Corporate  Corporate   
                                5108  Corporate LATC  Corporate  Corporate   
                                5109  Corporate LATC  Corporate  Corporate   
                                5110  Corporate LATC  Corporate  Corporate   
                                5111  Corporate LATC  Corporate  Corporate   
                                5112  Corporate LATC  Corporate  Corporate   
                                5113  Corporate LATC  Corporate  Corporate   
                                5114  Corporate LATC  Corporate  Corporate   
                                5115  Corporate LATC  Corporate  Corporate   
                                5116  Corporate LATC  Corporate  Corporate   
                                5117  Corporate LATC  Corporate  Corporate   
                                5118  Corporate LATC  Corporate  Corporate   
                                5266  Corporate LATC  Corporate    Service   
                                5267  Corporate LATC  Corporate    Service   
                                5268  Corporate LATC  Corporate    Service   
      Employee Benefits         394   Corporate LATC  Corporate  Corporate   
                                395   Corporate LATC  Corporate  Corporate   
                                396   Corporate LATC  Corporate  Corporate   
                                397   Corporate LATC  Corporate  Corporate   
                                398   Corporate LATC  Corporate  Corporate   
                                399   Corporate LATC  Corporate  Corporate   

                  

In [10]:
# Quick check about how many unique combination with brnid and fdb_commonname do we have in grouped
combination_check = df_1.groupby(['brnid', 'fdb_commonname']).size().reset_index()
# Print the result
print(combination_check)

     brnid             fdb_commonname     0
0      200  Building Repair and Maint    19
1      200          Employee Benefits  4772
2      200                 Rent/Lease   370
3      200                  Utilities    24
4      200           Vacation Expense   147
..     ...                        ...   ...
403    510           Delivery Expense   810
404    510          Employee Benefits   974
405    510                 Rent/Lease    36
406    510                  Utilities    81
407    510           Vacation Expense    25

[408 rows x 3 columns]


#### Filtered_df and Detect the potential Anomalies

In [11]:
# Suppress the SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [12]:
# The goal is to check the anomaly for amtdb , amtcr, and amtbalance for each unique group. 

# I make a list to store all anomalies
all_anomalies = []

# Iterate through each unique combination of brnid and fdb_commonname
for _, row in combination_check.iterrows():
    brnid_value = row['brnid']
    fdb_commonname_value = row['fdb_commonname']
    
    # Filter the data for this specific combination
    df_filtered = grouped_df_1[(grouped_df_1['brnid'] == brnid_value) & 
                               (grouped_df_1['fdb_commonname'] == fdb_commonname_value)]
    
    print(f"Processing brnid: {brnid_value}, fdb_commonname: {fdb_commonname_value}")
    
    # Columns we want to look through
    columns = ['amtdb', 'amtcr', 'amtbalance']
    
    # Apply Isolation Forest for each column
    for column in columns:
        # Build isolation forest
        iso_forest = IsolationForest(contamination=0.00001, random_state=42)
        
        # Fit the model using the filtered data
        iso_forest.fit(df_filtered[[column]])
        
        # Predict anomalies (-1 indicates an anomaly, 1 indicates a normal point)
        anomaly_column = f'{column} Anomaly'  # Create a new column for anomaly detection
        df_filtered[anomaly_column] = iso_forest.predict(df_filtered[[column]])

    # If one of the three variables has -1 (anomaly), list the row
    result_filtered_detection = df_filtered[
        (df_filtered['amtdb Anomaly'] == -1) |
        (df_filtered['amtcr Anomaly'] == -1) |
        (df_filtered['amtbalance Anomaly'] == -1)
    ].reset_index(drop=True)
    
    # Append the result to the list of all anomalies
    if not result_filtered_detection.empty:
        all_anomalies.append(result_filtered_detection)
    
# Combine all results into a single DataFrame
combined_anomalies_df = pd.concat(all_anomalies, ignore_index=True)

# Display the combined result
print("Combined anomalies DataFrame:")
print(combined_anomalies_df)

Processing brnid: 200, fdb_commonname: Building Repair and Maint
Processing brnid: 200, fdb_commonname: Employee Benefits
Processing brnid: 200, fdb_commonname: Rent/Lease
Processing brnid: 200, fdb_commonname: Utilities
Processing brnid: 200, fdb_commonname: Vacation Expense
Processing brnid: 210, fdb_commonname: Building Repair and Maint
Processing brnid: 210, fdb_commonname: Delivery Expense
Processing brnid: 210, fdb_commonname: Employee Benefits
Processing brnid: 210, fdb_commonname: Policy Adjustments
Processing brnid: 210, fdb_commonname: Rent/Lease
Processing brnid: 210, fdb_commonname: Supplies - Collected
Processing brnid: 210, fdb_commonname: Utilities
Processing brnid: 210, fdb_commonname: Vacation Expense
Processing brnid: 211, fdb_commonname: Building Repair and Maint
Processing brnid: 211, fdb_commonname: Delivery Expense
Processing brnid: 211, fdb_commonname: Employee Benefits
Processing brnid: 211, fdb_commonname: Policy Adjustments
Processing brnid: 211, fdb_commonnam

In [15]:
# Filter and group for 'amtdb Anomaly'
amtdb_anomaly_df = combined_anomalies_df[combined_anomalies_df['amtdb Anomaly'] == -1].reset_index(drop=True)
print("AMTDB Anomaly DataFrame shape:", amtdb_anomaly_df.shape)
grouped_counts_amtdb = amtdb_anomaly_df.groupby(['year', 'month']).size().reset_index(name='count')
print("Grouped Counts for AMTDB Anomaly:")
print(grouped_counts_amtdb)

# Filter and group for 'amtcr Anomaly'
amtcr_anomaly_df = combined_anomalies_df[combined_anomalies_df['amtcr Anomaly'] == -1].reset_index(drop=True)
print("AMTCR Anomaly DataFrame shape:", amtcr_anomaly_df.shape)
grouped_counts_amtcr = amtcr_anomaly_df.groupby(['year', 'month']).size().reset_index(name='count')
print("Grouped Counts for AMTCR Anomaly:")
print(grouped_counts_amtcr)

# Filter and group for 'amtbalance Anomaly'
amtbalance_anomaly_df = combined_anomalies_df[combined_anomalies_df['amtbalance Anomaly'] == -1].reset_index(drop=True)
print("AMTBALANCE Anomaly DataFrame shape:", amtbalance_anomaly_df.shape)
grouped_counts_amtbalance = amtbalance_anomaly_df.groupby(['year', 'month']).size().reset_index(name='count')
print("Grouped Counts for AMTBALANCE Anomaly:")
print(grouped_counts_amtbalance)





AMTDB Anomaly DataFrame shape: (307, 18)
Grouped Counts for AMTDB Anomaly:
    year  month  count
0   2023      1     41
1   2023      2      8
2   2023      3     10
3   2023      4     13
4   2023      5     13
5   2023      6     12
6   2023      7      6
7   2023      8     17
8   2023      9     10
9   2023     10      4
10  2023     11     10
11  2023     12     13
12  2024      1     15
13  2024      2     28
14  2024      3     22
15  2024      4     20
16  2024      5     21
17  2024      6     44
AMTCR Anomaly DataFrame shape: (307, 18)
Grouped Counts for AMTCR Anomaly:
    year  month  count
0   2023      1      8
1   2023      2     32
2   2023      3     21
3   2023      4      9
4   2023      5     10
5   2023      6     12
6   2023      7      6
7   2023      8     12
8   2023      9      8
9   2023     10      3
10  2023     11      9
11  2023     12      9
12  2024      1     16
13  2024      2     54
14  2024      3     28
15  2024      4     18
16  2024      5     31